In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc"><h2>--- Day 9: Stream Processing ---</h2><p>A large stream blocks your path. According to the locals, it's not safe to <span title="&quot;Don't cross the streams!&quot;, they yell, even though there's only one. They seem to think they're hilarious.">cross the stream</span> at the moment because it's full of <em>garbage</em>. You look down at the stream; rather than water, you discover that it's a <em>stream of characters</em>.</p>
<p>You sit for a while and record part of the stream (your puzzle input). The characters represent <em>groups</em> - sequences that begin with <code>{</code> and end with <code>}</code>. Within a group, there are zero or more other things, separated by commas: either another <em>group</em> or <em>garbage</em>. Since groups can contain other groups, a <code>}</code> only closes the <em>most-recently-opened unclosed group</em> - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.</p>
<p>Sometimes, instead of a group, you will find <em>garbage</em>. Garbage begins with <code>&lt;</code> and ends with <code>&gt;</code>. Between those angle brackets, almost any character can appear, including <code>{</code> and <code>}</code>. <em>Within</em> garbage, <code>&lt;</code> has no special meaning.</p>
<p>In a futile attempt to clean up the garbage, some program has <em>canceled</em> some of the characters within it using <code>!</code>: inside garbage, <em>any</em> character that comes after <code>!</code> should be <em>ignored</em>, including <code>&lt;</code>, <code>&gt;</code>, and even another <code>!</code>.</p>
<p>You don't see any characters that deviate from these rules.  Outside garbage, you only find well-formed groups, and garbage always terminates according to the rules above.</p>
<p>Here are some self-contained pieces of garbage:</p>
<ul>
<li><code>&lt;&gt;</code>, empty garbage.</li>
<li><code>&lt;random characters&gt;</code>, garbage containing random characters.</li>
<li><code>&lt;&lt;&lt;&lt;&gt;</code>, because the extra <code>&lt;</code> are ignored.</li>
<li><code>&lt;{!&gt;}&gt;</code>, because the first <code>&gt;</code> is canceled.</li>
<li><code>&lt;!!&gt;</code>, because the second <code>!</code> is canceled, allowing the <code>&gt;</code> to terminate the garbage.</li>
<li><code>&lt;!!!&gt;&gt;</code>, because the second <code>!</code> and the first <code>&gt;</code> are canceled.</li>
<li><code>&lt;{o"i!a,&lt;{i&lt;a&gt;</code>, which ends at the first <code>&gt;</code>.</li>
</ul>
<p>Here are some examples of whole streams and the number of groups they contain:</p>
<ul>
<li><code>{}</code>, <code>1</code> group.</li>
<li><code>{{{}}}</code>, <code>3</code> groups.</li>
<li><code>{{},{}}</code>, also <code>3</code> groups.</li>
<li><code>{{{},{},{{}}}}</code>, <code>6</code> groups.</li>
<li><code>{&lt;{},{},{{}}&gt;}</code>, <code>1</code> group (which itself contains garbage).</li>
<li><code>{&lt;a&gt;,&lt;a&gt;,&lt;a&gt;,&lt;a&gt;}</code>, <code>1</code> group.</li>
<li><code>{{&lt;a&gt;},{&lt;a&gt;},{&lt;a&gt;},{&lt;a&gt;}}</code>, <code>5</code> groups.</li>
<li><code>{{&lt;!&gt;},{&lt;!&gt;},{&lt;!&gt;},{&lt;a&gt;}}</code>, <code>2</code> groups (since all but the last <code>&gt;</code> are canceled).</li>
</ul>
<p>Your goal is to find the total score for all groups in your input. Each group is assigned a <em>score</em> which is one more than the score of the group that immediately contains it. (The outermost group gets a score of <code>1</code>.)</p>
<ul>
<li><code>{}</code>, score of <code>1</code>.</li>
<li><code>{{{}}}</code>, score of <code>1 + 2 + 3 = 6</code>.</li>
<li><code>{{},{}}</code>, score of <code>1 + 2 + 2 = 5</code>.</li>
<li><code>{{{},{},{{}}}}</code>, score of <code>1 + 2 + 3 + 3 + 3 + 4 = 16</code>.</li>
<li><code>{&lt;a&gt;,&lt;a&gt;,&lt;a&gt;,&lt;a&gt;}</code>, score of <code>1</code>.</li>
<li><code>{{&lt;ab&gt;},{&lt;ab&gt;},{&lt;ab&gt;},{&lt;ab&gt;}}</code>, score of <code>1 + 2 + 2 + 2 + 2 = 9</code>.</li>
<li><code>{{&lt;!!&gt;},{&lt;!!&gt;},{&lt;!!&gt;},{&lt;!!&gt;}}</code>, score of <code>1 + 2 + 2 + 2 + 2 = 9</code>.</li>
<li><code>{{&lt;a!&gt;},{&lt;a!&gt;},{&lt;a!&gt;},{&lt;ab&gt;}}</code>, score of <code>1 + 2 = 3</code>.</li>
</ul>
<p><em>What is the total score</em> for all groups in your input?</p>
</article>


In [2]:
tests = [
    {
        "name": "Example 1",
        "s": "{}",
        "expected": 1,
    },
    {
        "name": "Example 2",
        "s": "{{{}}}",
        "expected": 6,
    },
    {
        "name": "Example 3",
        "s": "{{},{}}",
        "expected": 5,
    },
    {
        "name": "Example 4",
        "s": "{{{},{},{{}}}}",
        "expected": 16,
    },
    {
        "name": "Example 5",
        "s": "{<a>,<a>,<a>,<a>}",
        "expected": 1,
    },
    {
        "name": "Example 6",
        "s": "{{<ab>},{<ab>},{<ab>},{<ab>}}",
        "expected": 9,
    },
    {
        "name": "Example 7",
        "s": "{{<!!>},{<!!>},{<!!>},{<!!>}}",
        "expected": 9,
    },
    {
        "name": "Example 8",
        "s": "{{<a!>},{<a!>},{<a!>},{<ab>}}",
        "expected": 3,
    },
]


def sum_score_groupes(s: str) -> int:
    (
        level,
        group_sum,
    ) = (
        0,
        0,
    )
    i, n = 0, len(s)

    while i < n:
        if s[i] == "<":
            while i < n and s[i] != ">":
                i += 2 if s[i] == "!" else 1
        elif s[i] == "{":
            level += 1
            group_sum += level
            i += 1
        elif s[i] == "}":
            level -= 1
            i += 1
        else:
            i += 1

    return group_sum


run_tests_params(sum_score_groupes, tests)


Test Example 1 passed, for sum_score_groupes.
Test Example 2 passed, for sum_score_groupes.
Test Example 3 passed, for sum_score_groupes.
Test Example 4 passed, for sum_score_groupes.
Test Example 5 passed, for sum_score_groupes.
Test Example 6 passed, for sum_score_groupes.
Test Example 7 passed, for sum_score_groupes.
Test Example 8 passed, for sum_score_groupes.
Success


In [3]:
with open("../input//day9.txt") as f:
    puzzle = f.read()

print(f"Part I: {sum_score_groupes(puzzle)}")

Part I: 10050


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>10050</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, you're ready to remove the garbage.</p>
<p>To prove you've removed it, you need to count all of the characters within the garbage.  The leading and trailing <code>&lt;</code> and <code>&gt;</code> don't count, nor do any canceled characters or the <code>!</code> doing the canceling.</p>
<ul>
<li><code>&lt;&gt;</code>, <code>0</code> characters.</li>
<li><code>&lt;random characters&gt;</code>, <code>17</code> characters.</li>
<li><code>&lt;&lt;&lt;&lt;&gt;</code>, <code>3</code> characters.</li>
<li><code>&lt;{!&gt;}&gt;</code>, <code>2</code> characters.</li>
<li><code>&lt;!!&gt;</code>, <code>0</code> characters.</li>
<li><code>&lt;!!!&gt;&gt;</code>, <code>0</code> characters.</li>
<li><code>&lt;{o"i!a,&lt;{i&lt;a&gt;</code>, <code>10</code> characters.</li>
</ul>
<p><em>How many non-canceled characters are within the garbage</em> in your puzzle input?</p>
</article>

</main>


In [27]:
tests_count_garbage = [
    {
        "name": "Example 1",
        "s": "{}",
        "expected": 0,
    },
    {
        "name": "Example 2",
        "s": "{{{}}}",
        "expected": 0,
    },
    {
        "name": "Example 3",
        "s": "{{},{}}",
        "expected": 0,
    },
    {
        "name": "Example 4",
        "s": "{{{},{},{{}}}}",
        "expected": 0,
    },
    {
        "name": "Example 5",
        "s": "{<a>,<a>,<a>,<a>}",
        "expected": 4,
    },
    {
        "name": "Example 6",
        "s": "{{<ab>},{<ab>},{<ab>},{<ab>}}",
        "expected": 8,
    },
    {
        "name": "Example 7",
        "s": "{{<!!>},{<!!>},{<!!>},{<!!>}}",
        "expected": 0,
    },
    {
        "name": "Example 8",
        "s": "{{<a!>},{<a!>},{<a!>},{<ab>}}",
        "expected": 17,
    },
]


def count_garbage(s: str) -> int:
    garbage, i, n = 0, 0, len(s)

    while i < n:
        ch = s[i]
        i += 1
        if ch == "<":
            while i < n and s[i] != ">":
                garbage += 0 if s[i] == "!" else 1
                i += 2 if s[i] == "!" else 1

    return garbage


run_tests_params(count_garbage, tests_count_garbage)


Test Example 1 passed, for count_garbage.
Test Example 2 passed, for count_garbage.
Test Example 3 passed, for count_garbage.
Test Example 4 passed, for count_garbage.
Test Example 5 passed, for count_garbage.
Test Example 6 passed, for count_garbage.
Test Example 7 passed, for count_garbage.
Test Example 8 passed, for count_garbage.
Success


In [28]:
print(f"Part II: {count_garbage(puzzle)}")

Part II: 4482


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4482</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>
